In [1]:
import numpy as np
import pandas as pd

In [2]:
import sqlite3

In [7]:
conn = sqlite3.connect('indicators_db.sqlite3')

In [8]:
with sqlite3.connect('indicators_db.sqlite3') as conn:
    indicators = pd.read_sql('SELECT * FROM sqlite_master', conn)

In [9]:
indicators

,type,name,tbl_name,rootpage,sql
0,table,org_indicators_2019,org_indicators_2019,2,CREATE TABLE org_indicators_2019 (\r\n\tID BIG...
1,table,org_indicators_2020_2022,org_indicators_2020_2022,490,CREATE TABLE org_indicators_2020_2022 (\r\n\tI...
2,view,v_org_indicators_all,v_org_indicators_all,0,CREATE VIEW v_org_indicators_all AS\r\nselect ...


In [12]:
with sqlite3.connect('indicators_db.sqlite3') as conn:
    indicators = pd.read_sql('SELECT * FROM org_indicators_2020_2022', conn)

In [13]:
indicators

,ID,YEAR,INDICATOR_NAME,VALUE
0,20988,2020,Выручка,29337447
1,20988,2020,Оценка на Яндекс,2
2,20988,2020,Налоги,720554
3,20988,2020,Оценка на 2GIS,3
4,24828,2020,Выручка,36633885
...,...,...,...,...
119995,339278,2022,Налоги,339380
119996,23511,2022,Число сотрудников,18
119997,23511,2022,Налоги,764187
119998,23511,2022,Выручка,22006160
